# Tests

## Test labels rearranging

In [15]:
import importlib
import numpy as np
import tensorflow as tf

from utils import is_equal
from utils.med_utils import label_utils

importlib.reload(label_utils)

labels  = [None, 'lung_left', 'lung_right', 'heart']
mapping = [None, 'heart', 'lung_left', 'lung_right']
one_hot_labels = labels[1:]

array = np.array([
    [1, 2, 1],
    [0, 0, 2],
    [3, 1, 2]
])
one_hot = np.equal(
    np.expand_dims(array, -1), np.arange(array.max() + 1).reshape([1] * len(array.shape) + [-1])
)[..., 1:].astype(np.uint8)
sparse = tf.sparse.from_dense(array)
sparse_one_hot = tf.sparse.from_dense(one_hot)

target = np.array([
    [2, 3, 2],
    [0, 0, 3],
    [1, 2, 3]
])

print(is_equal(target, label_utils.rearrange_labels(array, labels, mapping, is_one_hot = False))[1])
print(is_equal(target, label_utils.rearrange_labels(one_hot, one_hot_labels, mapping, is_one_hot = True))[1])
print(is_equal(
    target, tf.sparse.to_dense(label_utils.rearrange_labels(sparse, labels, mapping, is_one_hot = False))
)[1])
print(is_equal(
    target, np.argmax(tf.sparse.to_dense(
        label_utils.rearrange_labels(sparse_one_hot, one_hot_labels, mapping, is_one_hot = True)
    ), axis = -1).astype(np.int32)
)[1])


_ = """
array   = np.random.randint(0, len(labels), size = (256, 256, 256))
one_hot = np.equal(np.expand_dims(array, -1), np.arange(len(labels)).reshape([1] * len(array.shape) + [-1]))

%timeit rearrange_labels_one_hot(one_hot, labels, mapping)
%timeit rearrange_labels_one_hot(np.equal(np.expand_dims(array, -1), np.arange(len(labels)).reshape([1] * len(array.shape) + [-1])), labels, mapping)
%timeit rearrange_labels_dense(array, labels, mapping)
#%timeit rearrange_labels_vectorize(array, labels, mapping)
"""

Value are equals !
Value are equals !
Value are equals !
Value are equals !


## Test loss averaging

In [12]:
import numpy as np
import tensorflow as tf

from utils import compute_centroids

def average_loss(losses, batch_size, mode = 'macro', batch_idx = None, ids = None):
    if batch_size == 1:
        if mode == 'macro':
            return tf.reduce_mean(losses, axis = -1, keepdims = True)
        elif mode == 'micro':
            return tf.reduce_mean(tf.squeeze(compute_centroids(
                tf.expand_dims(losses, axis = 1), tf.cast(ids, tf.int32)
            )[1], axis = -1), axis = -1, keepdims = True)
    else:
        if mode == 'macro':
            return tf.squeeze(compute_centroids(
                tf.expand_dims(losses, axis = 1), tf.cast(batch_idx, tf.int32)
            )[1], axis = -1)
        else:
            n_labels    = tf.reduce_max(ids)
            indexes     = tf.cast(batch_idx * n_labels + ids, tf.int32)
            loss_label_batch_idx, loss_per_label_per_batch = compute_centroids(
                tf.expand_dims(losses, axis = 1), indexes
            )

            loss_batch_idx = loss_label_batch_idx // n_labels
            return tf.squeeze(compute_centroids(
                loss_per_label_per_batch, loss_batch_idx
            )[1], axis = 1)

losses = tf.cast([1, 2, 1.5, 2.5, 3, 4.5, 4, 3.5], tf.float32)
ids    = tf.cast([1, 1, 2, 2, 3, 2, 3, 2], tf.int32)
batch  = tf.cast([1, 1, 1, 1, 2, 2, 2, 2], tf.int32)

print(np.all(average_loss(losses, 1, 'macro', batch, ids) == np.mean(losses)))
print(np.all(average_loss(losses, 1, 'micro', batch, ids) == np.mean([1.5, 3, 3.5])))
print(np.all(average_loss(losses, 2, 'macro', batch, ids) == np.mean([[1, 2, 1.5, 2.5], [3, 3.5, 4, 4.5]], axis = -1)))
print(np.all(average_loss(losses, 2, 'macro', batch, ids) == np.mean([[1.5, 2], [3.5, 4]], axis = -1)))

True
True
True
True


## Test sparse argmax

In [2]:
import tensorflow as tf

mask = tf.sparse.SparseTensor(
    indices = [
        [1, 1, 1],
        [2, 1, 1],
        [2, 2, 1],
        [1, 2, 2],
        [2, 0, 2],
        [3, 1, 2]
    ],
    values = [1] * 6,
    dense_shape = (4, 4, 3)
)
mask = tf.sparse.reorder(mask)
mask = tf.sparse.expand_dims(mask, axis = 0)

res1 = tf.argmax(tf.sparse.to_dense(mask), axis = -1, output_type = tf.int32)
res2 = tf.tensor_scatter_nd_update(
    tf.zeros(mask.dense_shape[:-1], dtype = tf.int32),
    mask.indices[:, :-1],
    tf.cast(mask.indices[:, -1], tf.int32)
)
print(tf.reduce_all(res1 == res2))

tf.Tensor(True, shape=(), dtype=bool)


## Test `pad_or_crop`

In [1]:
import importlib

import tensorflow as tf

from utils import is_equal
from utils.med_utils import pre_processing

importlib.reload(pre_processing)

image = tf.reshape(tf.range(16), [4, 4])

print(image)

print('\nCropping tests')
crop_shape = (2, 2)
print(is_equal(
    image[:2, :2], pre_processing.pad_or_crop(image, crop_shape, crop_mode = 'start')
)[1])
print(is_equal(
    image[1:3, 1:3], pre_processing.pad_or_crop(image, crop_shape, crop_mode = 'center')
)[1])
print(is_equal(
    image[-2:, -2:], pre_processing.pad_or_crop(image, crop_shape, crop_mode = 'end')
)[1])

print('\nPadding tests')
pad_shape = (6, 6)
print(is_equal(
    tf.pad(image, [(2, 0)] * 2), pre_processing.pad_or_crop(image, pad_shape, pad_mode = 'before', pad_value = 0)
)[1])
print(is_equal(
    tf.pad(image, [(1, 1)] * 2), pre_processing.pad_or_crop(image, pad_shape, pad_mode = 'even', pad_value = 0)
)[1])
print(is_equal(
    tf.pad(image, [(0, 2)] * 2), pre_processing.pad_or_crop(image, pad_shape, pad_mode = 'after', pad_value = 0)
)[1])

print('\nCropping + padding tests')
crop_pad_shape = (6, 2)
print(is_equal(tf.pad(image[:, 1:3], [(1, 1), (0, 0)]), pre_processing.pad_or_crop(
    image, crop_pad_shape, crop_mode = 'center', pad_mode = 'even', pad_value = 0
))[1])

2023-05-23 08:42:58.125256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 08:42:58.225513: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 08:42:58.250711: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

tf.Tensor(
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]], shape=(4, 4), dtype=int32)

Cropping tests
Value are equals !
Value are equals !
Value are equals !

Padding tests
Value are equals !
Value are equals !
Value are equals !

Cropping + padding tests
Value are equals !


## Test `crop_then_reshape`

In [1]:
import importlib
import numpy as np
import tensorflow as tf

from utils import is_equal
from datasets import get_dataset
from utils.med_utils import pre_processing, resample_volume, load_medical_image, load_medical_seg

try:
    tf.config.set_visible_devices([], 'GPU')
except:
    pass

dataset = get_dataset('total_segmentator')

print('Dataset length : {}'.format(len(dataset)))

2023-05-24 10:02:59.878059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 10:03:01.035539: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 10:03:01.555494: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

Loading dataset total_segmentator...
Dataset length : 1203


In [3]:
importlib.reload(pre_processing)

# All the below tests should alos work if the image has a 4th channel axis (uncomment the below line to test it)
row = dataset.iloc[0]
img, voxel_dims = load_medical_image(row['images'])
#img = tf.expand_dims(img, axis = -1)
print('Image shape : {} - voxel dims : {}'.format(img.shape, voxel_dims))

print('\nTest cropping')
target_shape = (128, 128, 32)
target_voxel = voxel_dims

img2 = pre_processing.crop_then_reshape(img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel)
tar2 = pre_processing.pad_or_crop(img, target_shape = target_shape)
print(is_equal(target_shape, img2.shape[:len(target_shape)])[1])
print(is_equal(tar2, img2)[1])

print('\nTest padding')
target_shape = (256, 256, 32)
target_voxel = voxel_dims
img3 = pre_processing.crop_then_reshape(img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel)
tar3 = pre_processing.pad_or_crop(img, target_shape = target_shape)
print(is_equal(target_shape, img3.shape[:len(target_shape)])[1])
print(is_equal(tar3, img3)[1])

print('\nTest cropping then resizing')
target_shape = (128, 128, 32)
target_voxel = (0.5, 0.5, 0.5)

img4 = pre_processing.crop_then_reshape(img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel)
tar4, new_voxel = resample_volume(
    pre_processing.pad_or_crop(img, target_shape = np.round(np.array(target_shape) / 3).astype(np.int32)),
    voxel_dims   = voxel_dims,
    target_shape = target_shape
)
print(is_equal(target_shape, img4.shape[:len(target_shape)])[1])
print(is_equal(target_voxel, new_voxel, max_err = 0.1)[1])
print(is_equal(tar4, img4)[1])

print('\nTest padding then resizing')
target_shape = (128, 128, 32)
target_voxel = (3, 3, 3)

img5 = pre_processing.crop_then_reshape(img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel)
tar5, new_voxel = resample_volume(
    pre_processing.pad_or_crop(img, target_shape = np.array(target_shape) * 2),
    voxel_dims   = voxel_dims,
    target_shape = target_shape
)
print(is_equal(target_shape, img5.shape[:len(target_shape)])[1])
print(is_equal(target_voxel, new_voxel, max_err = 1e-3)[1])
print(is_equal(tar5, img5)[1])

print('\nTest simple frame cropping')
target_shape = (-1, -1, 32)
target_voxel = voxel_dims

img6 = pre_processing.crop_then_reshape(
    img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel, crop_mode = 'start'
)
tar6, new_voxel = img[:, :, :target_shape[2]], target_voxel
print(is_equal(tar6.shape, img6.shape)[1])
print(is_equal(target_voxel, new_voxel, max_err = 0.1)[1])
print(is_equal(tar6, img6)[1])

print('\nTest simple resizing')
# in this scenario, the target should be a resized version of the 64 first frames
# the resized should have a shape half of the original shape, as the new voxel dim is 2 times higher
# if each voxel is 2 times bigger, we should have a volume 2 times smaller to cover the same space in the 3D-world space
# It is the reason why we take the 64 first frames then resize them to 32
target_shape = (-1, -1, 32)
target_voxel = (3, 3, 3)

img7 = pre_processing.crop_then_reshape(
    img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel, crop_mode = 'start'
)
tar7, new_voxel = resample_volume(
    img[:, :, :target_shape[2] * 2],
    voxel_dims   = voxel_dims,
    target_shape = [s if s > 0 else img.shape[i] // 2 for i, s in enumerate(target_shape)]
)
print(is_equal(tar7.shape, img7.shape)[1])
print(is_equal(target_voxel, new_voxel, max_err = 0.1)[1])
print(is_equal(tar7, img7)[1])

print('\nTest simple resize to multiple')
target_shape = (-1, -1, 32)
target_voxel = voxel_dims
multiples    = np.array([32, 32, 32])

img8 = pre_processing.crop_then_reshape(
    img, voxel_dims, target_shape = target_shape, target_voxel_dims = target_voxel, crop_mode = 'start',
    multiple_shape = multiples
)
tar8, new_voxel = resample_volume(
    img[:, :, :target_shape[2]], voxel_dims, target_voxel_dims = voxel_dims,
    target_shape = [
        img.shape[0] // multiples[0] * multiples[0],
        img.shape[1] // multiples[1] * multiples[1],
        target_shape[2] // multiples[2] * multiples[2]
    ]
)
print(is_equal(tar8.shape, img8.shape)[1])
print(is_equal(target_voxel, new_voxel, max_err = 0.1)[1])
print(is_equal(tar8, img8)[1])

Image shape : (249, 188, 213) - voxel dims : [1.4999999 1.5       1.4999999]

Test cropping
Value are equals !
Value are equals !

Test padding
Value are equals !
Value are equals !

Test cropping then resizing
Value are equals !
Value are equals !
Value are equals !

Test padding then resizing
Value are equals !
Value are equals !
Value are equals !

Test simple frame cropping
Value are equals !
Value are equals !
Value are equals !

Test simple resizing
Value are equals !
Value are equals !
Value are equals !

Test simple resize to multiple
Value are equals !
Value are equals !
Value are equals !


## Test Clustering Loss

In [1]:
import os
import importlib
import numpy as np
import tensorflow as tf

from custom_train_objects.losses import ge2e_seg_loss

try:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')
except:
    pass

In [17]:
importlib.reload(ge2e_seg_loss)

loss = ge2e_seg_loss.GE2ESegLoss()

mask = tf.sparse.SparseTensor(
    indices = [
        [1, 1, 1],
        [2, 1, 1],
        [2, 2, 1],
        [1, 2, 2],
        [2, 0, 2],
        [3, 1, 2]
    ],
    values = [1] * 6,
    dense_shape = (4, 4, 3)
)
mask = tf.sparse.reorder(mask)
mask = tf.sparse.expand_dims(mask, axis = 0)
embeddings = np.zeros((1, 4, 4, 2))
for i, j in mask.indices[:, 1:-1].numpy(): embeddings[:, i, j] = [i, j]
embeddings = tf.cast(embeddings, tf.float32)

fore_emb, fore_centr, fore_centr_ids = loss.compute_foreground_centroids(mask, embeddings)
print(fore_emb)
print(fore_centr_ids)
print(fore_centr)

back_emb, back_centr, _ = loss.compute_background_centroid(mask, embeddings)
print(back_emb)
print(back_centr)
print(loss(mask, embeddings))

tf.Tensor(
[[1. 1.]
 [1. 2.]
 [2. 0.]
 [2. 1.]
 [2. 2.]
 [3. 1.]], shape=(6, 2), dtype=float32)
tf.Tensor([0 1 1 0 0 1], shape=(6,), dtype=int32)
tf.Tensor(
[[1.6666666 1.3333334]
 [2.        1.       ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]], shape=(10, 2), dtype=float32)
tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(0.9176704, shape=(), dtype=float32)


## Test Dice Loss

In [ ]:
import os
import importlib
import numpy as np
import tensorflow as tf

from custom_train_objects.losses import dice_loss
from utils.med_utils import load_medical_image, load_medical_seg

try:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')
except:
    pass

path  = '/storage/Totalsegmentator_dataset/s0001'

#image = tf.expand_dims(load_medical_image(os.path.join(path, 'ct.nii.gz'))[0], axis = 0)
mask  = tf.sparse.expand_dims(load_medical_seg(os.path.join(path, 'masks.npz'))[0], axis = 0)

print('Image shape : {} - mask shape : {}'.format((), mask.shape))

In [ ]:
importlib.reload(dice_loss)

loss = dice_loss.DiceLoss(skip_empty_frames = False, skip_empty_labels = True, smoothing = 0.01)

dense_mask = tf.sparse.to_dense(tf.cast(mask, tf.float32))

empty_labels = tf.sparse.reduce_sum(tf.sparse.reshape(mask, [1, -1, 104]), axis = 1) == 0
empty_labels = tf.cast(tf.reshape(empty_labels, [1, 1, 1, 1, 104]), tf.float32)

print(empty_labels)

print(loss(mask, dense_mask).numpy())
print(loss(mask, dense_mask * 0.5).numpy())
print(loss(mask, dense_mask + 0.25 * empty_labels).numpy())

In [ ]:
%timeit loss(mask, dense_mask + 0.25 * empty_labels)
%timeit loss(dense_mask, dense_mask + 0.25 * empty_labels)
%timeit loss(tf.sparse.to_dense(mask), dense_mask + 0.25 * empty_labels)
